In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import dynamicComputation as dc
import lengthPriors
import numpy as np
import scipy as sp
import scipy.signal
import segment_viz
%matplotlib widget

In [2]:
in_data = pd.read_csv('/Users/guichaoua 1/Nextcloud/Cardiac_response_to_live_music/Updated features/full_ARIs/2018_05_22_P1.csv')

In [3]:
plt.plot(in_data.time, in_data.ARI)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# First try

In [4]:
lengthPrior = lengthPriors.EmpiricalLengthPrior(range(5, 120), len(in_data.ARI))

In [5]:

def make_geo_dist(min_len, max_len, ratio):
    unscaled = [ratio**i for i in range(max_len-min_len)]
    factor = sum(unscaled)
    return {i:p/factor for i, p in zip(range(min_len, max_len), unscaled)}


lengthPrior = lengthPriors.DiscreteLengthPrior(len(in_data.ARI), make_geo_dist(5, 60, .8), 60)

In [6]:
est_noise = np.std(np.diff(in_data.ARI))/2

In [7]:
arcPrior = {
    # Gaussian priors on the parameters of ax^2 + bx + c
    'aMean': 0,
    'aStd': 00,
    'bMean': 00,
    'bStd': 00,
    'cMean': 330,
    'cStd': 15,
    'noiseStd': est_noise
}

In [8]:
post, post_bidim = dc.runAlphaBeta(in_data.ARI, arcPrior, lengthPrior, return2D=True)

In [9]:
f=segment_viz.plot_segment_with_signal(post, in_data.ARI, in_data.time, data_color='y', boundaries=in_data.time[post>.5])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Let's plot only the strong boundaries

In [10]:
fig=plt.figure(figsize=(12,8), dpi= 100)

plt.plot(in_data.time[:len(post)], in_data.ARI[:len(post)],'r')
# plt.xlim((0,500))
ax2 = plt.twinx()

high_post = post>.5
ax2.vlines(in_data.time[high_post],0,1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Try 2D plots

In [11]:
smoothed = sp.signal.convolve2d(post_bidim, np.ones((5,5)), mode="same")
for i in range(np.shape(smoothed)[0]):
    smoothed[i,i]=1

In [12]:
fig=plt.figure(figsize=(12,8), dpi= 100)

plt.imshow(smoothed)
plt.colorbar()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Do it again with other settings

We use a Uniform Prior $U([10,90])$

In [13]:
lengthPrior = lengthPriors.EmpiricalLengthPrior(range(10, 90), len(in_data.ARI))

In [14]:
est_noise = np.std(np.diff(in_data.ARI)/2)

In [15]:
arcPrior = {
    # Gaussian priors on the parameters of ax^2 + bx + c
    'aMean': 0,
    'aStd': 00,
    'bMean': 00,
    'bStd': 00,
    'cMean': 330,
    'cStd': 15,
    'noiseStd': est_noise
}

In [16]:
post2, post2_bidim = dc.runAlphaBeta(in_data.ARI, arcPrior, lengthPrior, return2D=True)

In [17]:
fig=plt.figure(figsize=(12,8), dpi= 100)

plt.plot(in_data.time[:len(post)], in_data.ARI[:len(post)],'r')
# plt.xlim((500,1000))
ax2 = plt.twinx()

# ax2.plot(in_data.time[:len(post)],post)
post2_smoothed = np.convolve([1,1,1,1,1],post2,mode='same')

ax2.plot(in_data.time,post2_smoothed)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Let's plot only the strong boundaries

In [18]:
fig=plt.figure(figsize=(12,8), dpi= 100)

plt.plot(in_data.time[:len(post)], in_data.ARI[:len(post)],'r')
# plt.xlim((0,500))
ax2 = plt.twinx()

high_post = post2_smoothed>.5
ax2.vlines(in_data.time[high_post],0,1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
sum(post2)



38.21470139788907

In [20]:
sum(post)

143.14722002635443

In [21]:
import scipy.stats

pdf = scipy.stats.norm.pdf([0.3,0.4,-1])
pdf

array([0.38138782, 0.36827014, 0.24197072])

# Try 2D plots

In [22]:
segment_viz.plot_segment_matrix(post2_bidim, smoothing=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
smoothed2 = sp.signal.convolve2d(post2_bidim, np.ones((5,5)), mode="same")
for i in range(np.shape(smoothed2)[0]):
    smoothed2[i,i]=1

In [24]:
fig, ax =plt.subplots(figsize=(12,8), dpi= 100)

im = ax.imshow(smoothed2, cmap=plt.get_cmap('hot'))
cbar = fig.colorbar(im)
cbar.set_label("Likelihood")
ax.set_ylabel('Start')
ax.set_xlabel('End')
ax.set_title('Posterior likelihood of segment')


fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
viz = segment_viz.plot_segment_raindrop(post2_bidim, lengthPrior.maxLength)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
viz2 = segment_viz.plot_segment_beams(post2_bidim, lengthPrior.maxLength)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
linear_viz = np.zeros((np.shape(post2_bidim)[0], lengthPrior.maxLength))
for i in range(np.shape(post2_bidim)[0]):
    for j in range(lengthPrior.maxLength):
        if i+j < np.shape(post2_bidim)[0]:
            for k in range(j):
                linear_viz[i+k,j] = linear_viz[i+k,j] + post2_bidim[i,i+j]

In [28]:
fig=plt.figure(figsize=(16,4), dpi= 100)

plt.imshow(np.transpose(linear_viz), cmap=plt.get_cmap('hot'), origin='lower', aspect=4)
plt.colorbar()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
linear_viz_point = np.zeros((np.shape(smoothed2)[0], lengthPrior.maxLength))
for i in range(np.shape(smoothed2)[0]):
    for j in range(lengthPrior.maxLength):
        if i+j < np.shape(smoothed2)[0]:
            for k in range(1):
                linear_viz_point[i+k,j] = linear_viz_point[i+k,j] + smoothed2[i,i+j]

In [30]:
fig=plt.figure(figsize=(16,4), dpi= 100)

plt.imshow(np.transpose(linear_viz_point), cmap=plt.get_cmap('hot'), origin='lower', aspect=4)
plt.colorbar()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
priorMarginals, prior_bidim = dc.runAlphaBeta([0 for _ in in_data.ARI], arcPrior, lengthPrior, return2D=True)

In [33]:
_=segment_viz.plot_segment_with_signal(priorMarginals,[0 for _ in in_data.ARI],in_data.time, smoothing=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
geo_prior = lengthPriors.DiscreteLengthPrior(len(in_data.ARI), make_geo_dist(5, 60, .8), 60)
geo_prior_marginals, geo_prior_bidim = dc.runAlphaBeta([0 for _ in in_data.ARI], arcPrior, geo_prior, return2D=True)

In [45]:
_=segment_viz.plot_segment_with_signal(geo_prior_marginals,[0 for _ in in_data.ARI],in_data.time, smoothing=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
uni_prior = lengthPriors.EmpiricalLengthPrior(list(range(10,15)),len(in_data.ARI))
uni_prior_marginals, uni_prior_bidim = dc.runAlphaBeta([0 for _ in in_data.ARI], arcPrior, uni_prior, return2D=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
_=segment_viz.plot_segment_with_signal(uni_prior_marginals,[0 for _ in in_data.ARI],in_data.time, smoothing=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
_ = segment_viz.plot_segment_raindrop(prior_bidim, max_length=120)
_ = segment_viz.plot_segment_raindrop(geo_prior_bidim, max_length=120)
_ = segment_viz.plot_segment_raindrop(uni_prior_bidim, max_length=120)

/Users/guichaoua 1/Nextcloud/Workspace/ArcV2/src/segment_viz.py:34: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=(16, 4), dpi=100)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
print(geo_prior.distrib.get(15)**2)
print(geo_prior.distrib.get(10)*geo_prior.distrib.get(20))

0.00046117291546446653
0.0004611729154644665


In [50]:
print(geo_prior.evalCond(-1,10) * geo_prior.evalCond(10,30))
print(geo_prior.evalCond(-1,15) * geo_prior.evalCond(15,30))

0.00023612053271780686
0.00023612053271780686


In [58]:
geo_prior = lengthPriors.DiscreteLengthPrior(30, make_geo_dist(11, 18, .8), 20)
geo_prior_marginals, geo_prior_bidim = dc.runAlphaBeta([0 for _ in range(30)], arcPrior, geo_prior, return2D=True)

In [59]:
_=segment_viz.plot_segment_with_signal(geo_prior_marginals,[0 for _ in range(30)],list(range(30)), smoothing=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
print(geo_prior_marginals)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.16334806 0.17131024 0.18671393 0.21409602 0.26453175 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         1.        ]
